# Task 1: Create a Prescription Parser using CRF
This task tests your ability to build a Doctor Prescription Parser with the help of CRF model

Your job is to build a Prescription Parser that takes a prescription (sentence) as an input and find / label the words in that sentence with one of the already pre-defined labels

### Problem: SEQUENCE PREDICTION - Label words in a sentence
#### Input : Doctor Prescription in the form of a sentence split into tokens
- Ex: Take 2 tablets once a day for 10 days

#### Output : FHIR Labels
- ('Take', 'Method')
- ('2', 'Qty') 
- ('tablets', 'Form')
- ('once', 'Frequency')
- ('a', 'Period') 
- ('day', 'PeriodUnit')
- ('for', 'FOR')
- ('10', 'Duration')
- ('days', 'DurationUnit') 

### Major Steps
- Install necessary library
- Import the libraries
- Create training data with labels
    - Split the sentence into tokens
    - Compute POS tags
    - Create triples
- Extract features
- Split the data into training and testing set
- Create CRF model
- Save the CRF model
- Load the CRF model
- Predict on test data
- Accuracy

#### Install necesaary library

In [46]:
pip install sklearn-crfsuite

DEPRECATION: Loading egg at /Users/omisesan/anaconda3/lib/python3.11/site-packages/endtoendmlgit-0.0.1-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/omisesan/anaconda3/lib/python3.11/site-packages/fonttools-4.55.2-py3.11-macosx-10.9-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for python-crfsuite: filename=python_crfsuite-0.9.11-cp311-cp311-macosx_10_9_x86_64.whl size=321438 sha256=8162de3607bf1fb11339e8a6b292830560849d4aa12b6548afdb36aa2726f07d
  Stored in directory: /Users/omisesan/Library/Cach

#### Import the necessary libraries

In [17]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/omisesan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [33]:
import spacy 

nlp = spacy.load('en_core_web_sm')
print(nlp.pipe_names)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [1]:
import sklearn_crfsuite
import joblib
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from sklearn_crfsuite import CRF, metrics
from sklearn.metrics import make_scorer,confusion_matrix
from pprint import pprint


### Input data (GIVEN)
#### Creating the inputs to the ML model in the following form:
- sigs --> ['take 3 tabs for 10 days']       INPUT SIG
- input_sigs --> [['take', '3', 'tabs', 'for', '10', 'days']]      TOKENS
- output_labels --> [['Method','Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]       LABELS

In [2]:
sigs = ["for 5 to 6 days", "inject 2 units", "x 2 weeks", "x 3 days", "every day", "every 2 weeks", "every 3 days", "every 1 to 2 months", "every 2 to 6 weeks", "every 4 to 6 days", "take two to four tabs", "take 2 to 4 tabs", "take 3 tabs orally bid for 10 days at bedtime", "swallow three capsules tid orally", "take 2 capsules po every 6 hours", "take 2 tabs po for 10 days", "take 100 caps by mouth tid for 10 weeks", "take 2 tabs after an hour", "2 tabs every 4-6 hours", "every 4 to 6 hours", "q46h", "q4-6h", "2 hours before breakfast", "before 30 mins at bedtime", "30 mins before bed", "and 100 tabs twice a month", "100 tabs twice a month", "100 tabs once a month", "100 tabs thrice a month", "3 tabs daily for 3 days then 1 tab per day at bed", "30 tabs 10 days tid", "take 30 tabs for 10 days three times a day", "qid q6h", "bid", "qid", "30 tabs before dinner and bedtime", "30 tabs before dinner & bedtime", "take 3 tabs at bedtime", "30 tabs thrice daily for 10 days ", "30 tabs for 10 days three times a day", "Take 2 tablets a day", "qid for 10 days", "every day", "take 2 caps at bedtime", "apply 3 drops before bedtime", "take three capsules daily", "swallow 3 pills once a day", "swallow three pills thrice a day", "apply daily", "apply three drops before bedtime", "every 6 hours", "before food", "after food", "for 20 days", "for twenty days", "with meals"]
input_sigs = [['for', '5', 'to', '6', 'days'], ['inject', '2', 'units'], ['x', '2', 'weeks'], ['x', '3', 'days'], ['every', 'day'], ['every', '2', 'weeks'], ['every', '3', 'days'], ['every', '1', 'to', '2', 'months'], ['every', '2', 'to', '6', 'weeks'], ['every', '4', 'to', '6', 'days'], ['take', 'two', 'to', 'four', 'tabs'], ['take', '2', 'to', '4', 'tabs'], ['take', '3', 'tabs', 'orally', 'bid', 'for', '10', 'days', 'at', 'bedtime'], ['swallow', 'three', 'capsules', 'tid', 'orally'], ['take', '2', 'capsules', 'po', 'every', '6', 'hours'], ['take', '2', 'tabs', 'po', 'for', '10', 'days'], ['take', '100', 'caps', 'by', 'mouth', 'tid', 'for', '10', 'weeks'], ['take', '2', 'tabs', 'after', 'an', 'hour'], ['2', 'tabs', 'every', '4-6', 'hours'], ['every', '4', 'to', '6', 'hours'], ['q46h'], ['q4-6h'], ['2', 'hours', 'before', 'breakfast'], ['before', '30', 'mins', 'at', 'bedtime'], ['30', 'mins', 'before', 'bed'], ['and', '100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'once', 'a', 'month'], ['100', 'tabs', 'thrice', 'a', 'month'], ['3', 'tabs', 'daily', 'for', '3', 'days', 'then', '1', 'tab', 'per', 'day', 'at', 'bed'], ['30', 'tabs', '10', 'days', 'tid'], ['take', '30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['qid', 'q6h'], ['bid'], ['qid'], ['30', 'tabs', 'before', 'dinner', 'and', 'bedtime'], ['30', 'tabs', 'before', 'dinner', '&', 'bedtime'], ['take', '3', 'tabs', 'at', 'bedtime'], ['30', 'tabs', 'thrice', 'daily', 'for', '10', 'days'], ['30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['take', '2', 'tablets', 'a', 'day'], ['qid', 'for', '10', 'days'], ['every', 'day'], ['take', '2', 'caps', 'at', 'bedtime'], ['apply', '3', 'drops', 'before', 'bedtime'], ['take', 'three', 'capsules', 'daily'], ['swallow', '3', 'pills', 'once', 'a', 'day'], ['swallow', 'three', 'pills', 'thrice', 'a', 'day'], ['apply', 'daily'], ['apply', 'three', 'drops', 'before', 'bedtime'], ['every', '6', 'hours'], ['before', 'food'], ['after', 'food'], ['for', '20', 'days'], ['for', 'twenty', 'days'], ['with', 'meals']]
output_labels = [['FOR', 'Duration', 'TO', 'DurationMax', 'DurationUnit'], ['Method', 'Qty', 'Form'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['EVERY', 'Period'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'Form', 'PO', 'BID', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'TID', 'PO'], ['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'PO', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'BY', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'AFTER', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Q46H'], ['Q4-6H'], ['Qty', 'PeriodUnit', 'BEFORE', 'WHEN'], ['BEFORE', 'Qty', 'M', 'AT', 'WHEN'], ['Qty', 'M', 'BEFORE', 'WHEN'], ['AND', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit', 'THEN', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'AT', 'WHEN'], ['Qty', 'Form', 'Duration', 'DurationUnit', 'TID'], ['Method', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Qty', 'TIMES', 'Period', 'PeriodUnit'], ['QID', 'Q6H'], ['BID'], ['QID'],['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Qty', 'Form', 'Frequency', 'DAILY', 'FOR', 'Duration', 'DurationUnit'], ['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Frequency', 'TIMES', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Period', 'PeriodUnit'], ['QID', 'FOR', 'Duration', 'DurationUnit'], ['EVERY', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['Method', 'Qty', 'Form', 'DAILY'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'DAILY'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['EVERY', 'Period', 'PeriodUnit'], ['BEFORE', 'FOOD'], ['AFTER', 'FOOD'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['WITH', 'FOOD']]

In [3]:
len(sigs), len(input_sigs) , len(output_labels)

(56, 56, 56)

### Creating a Tuples Maker method
Create the tuples as given below by writing a function **tuples_maker(input_sigs, output_labels)** and returns **output** as given below

Input(s): 
- input_sigs
- output_lables

Output:

[[('for', 'FOR'),
  ('5', 'Duration'),
  ('to', 'TO'),
  ('6', 'DurationMax'),
  ('days', 'DurationUnit')], [second sentence], ...]

In [13]:
def tuples_maker(input_sigs, output_labels):
    
    return[[(word, label) for word, label in zip(words, labels)] for words, labels in zip(input_sigs, output_labels)]

In [14]:
structured_data = tuples_maker(input_sigs, output_labels)
for sentence in structured_data:
        print(sentence)

[('for', 'FOR'), ('5', 'Duration'), ('to', 'TO'), ('6', 'DurationMax'), ('days', 'DurationUnit')]
[('inject', 'Method'), ('2', 'Qty'), ('units', 'Form')]
[('x', 'FOR'), ('2', 'Duration'), ('weeks', 'DurationUnit')]
[('x', 'FOR'), ('3', 'Duration'), ('days', 'DurationUnit')]
[('every', 'EVERY'), ('day', 'Period')]
[('every', 'EVERY'), ('2', 'Period'), ('weeks', 'PeriodUnit')]
[('every', 'EVERY'), ('3', 'Period'), ('days', 'PeriodUnit')]
[('every', 'EVERY'), ('1', 'Period'), ('to', 'TO'), ('2', 'PeriodMax'), ('months', 'PeriodUnit')]
[('every', 'EVERY'), ('2', 'Period'), ('to', 'TO'), ('6', 'PeriodMax'), ('weeks', 'PeriodUnit')]
[('every', 'EVERY'), ('4', 'Period'), ('to', 'TO'), ('6', 'PeriodMax'), ('days', 'PeriodUnit')]
[('take', 'Method'), ('two', 'Qty'), ('to', 'TO'), ('four', 'Qty'), ('tabs', 'Form')]
[('take', 'Method'), ('2', 'Qty'), ('to', 'TO'), ('4', 'Qty'), ('tabs', 'Form')]
[('take', 'Method'), ('3', 'Qty'), ('tabs', 'Form'), ('orally', 'PO'), ('bid', 'BID'), ('for', 'FOR'),

### Creating the triples_maker( ) for feature extraction
- input: tuples_maker_output
- output: 
[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')], [second sentence], ... ]

In [53]:
def triples_maker(structred_data):
    
    return [
        [(word, nltk.pos_tag([word])[0][1], label) for word, label in zip(words, labels)]
        for words, labels in zip(input_sigs, output_labels)
    ]

In [19]:
sample_data = triples_maker(structured_data)
sample_data

[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')],
 [('inject', 'NN', 'Method'), ('2', 'CD', 'Qty'), ('units', 'NNS', 'Form')],
 [('x', 'NN', 'FOR'),
  ('2', 'CD', 'Duration'),
  ('weeks', 'NNS', 'DurationUnit')],
 [('x', 'NN', 'FOR'),
  ('3', 'CD', 'Duration'),
  ('days', 'NNS', 'DurationUnit')],
 [('every', 'DT', 'EVERY'), ('day', 'NN', 'Period')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('3', 'CD', 'Period'),
  ('days', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('1', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('2', 'CD', 'PeriodMax'),
  ('months', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('4', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('

### Creating the features extractor method (GIVEN as a BASELINE)
#### The features used are:
- SOS, EOS, lowercase, uppercase, title, digit, postag, previous_tag, next_tag
#### Feel free to include more features

In [54]:
def token_to_features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

### Running the feature extractor on the training data 
- Feature extraction
- Train-test-split

In [73]:
from sklearn.model_selection import train_test_split

def get_features (doc) :
    return [token_to_features(doc, i) for i in range (len(doc) )]

def get_labels (doc) :
    return [label for (token, postag, label) in doc]

X = [get_features(doc) for doc in sample_data]

y = [get_labels(doc) for doc in sample_data]
X_train, X_test, Y_train, y_test = train_test_split(X, y, test_size=0.2 ,  random_state= 42)

### Training the CRF model with the features extracted using the feature extractor method


# Submit training data to the trainer
def train_crf_model(X_train, Y_train):
    # Set the parameters of the model
    crf = CRF(algorithm='lbfgs', c1=0.1, c2=10, max_iterations=50)
    crf.fit(X_train, Y_train)
    print("Training complete. Model results:")
    print(f"Number of training instances: {len(X_train)}")
    print(f"Number of features: {len(X_train[0]) if X_train else 0}")
    print(f"CRF Model Parameters: {crf.get_params()}")


train_crf_model(X_train, Y_train)

In [74]:
import pycrfsuite

In [75]:
trainer = pycrfsuite.Trainer(verbose=True)

for xseq, yseq in zip(X_train, Y_train):
    trainer.append(xseq, yseq)

trainer.set_params({
    'c1': 0.1,                   
    'c2': 0.01,                  
    'max_iterations': 1000,      
    'feature.possible_transitions': True
})


trainer.train('crf_model.crfsuite')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 1757
Seconds required: 0.003

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 1000
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 567.279176
Feature norm: 1.000000
Error norm: 116.778352
Active features: 1734
Line search trials: 1
Line search step: 0.006400
Seconds required for this iteration: 0.001

***** Iteration #2 *****
Loss: 385.155807
Feature norm: 5.077424
Error norm: 144.645872
Active features: 1489
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.001

***** Iteration #3 *****
Loss: 277.028174
Feature norm: 5.992673
Error norm: 62.587411
Active features: 1490
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration:

In [76]:
tagger = pycrfsuite.Tagger()
tagger.open('crf_model.crfsuite')

### Predicting the test data with the built model

In [77]:
y_pred = tagger.tag(X_test[0])
print(f'Predictions:', y_pred)

Predictions: ['FOR', 'Duration', 'TO', 'PeriodMax', 'PeriodUnit']


In [78]:
y_pred = tagger.tag(X_test[1])
print(f'Predictions:', y_pred)

Predictions: ['EVERY', 'Period', 'PeriodUnit']


In [79]:
y_pred = tagger.tag(X_test[2])
print(f'Predictions:', y_pred)

Predictions: ['QID']


In [80]:
y_pred = tagger.tag(X_test[3])
print(f'Predictions:', y_pred)

Predictions: ['Method', 'Qty', 'Form', 'TID', 'DAILY']


### Putting all the prediction logic inside a predict method

In [81]:
def predict(sig):
    """
    predict(sig)
    Purpose: Labels the given sig into corresponding labels
    @param sig. A Sentence  # A medical prescription sig written by a doctor
    @return     A list      # A list with predicted labels (first level of labeling)
    >>> predict('2 tabs every 4 hours')
    [['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit']]
    >>> predict('2 tabs with food')
    [['Qty', 'Form', 'WITH', 'FOOD']]
    >>> predict('2 tabs qid x 30 days')
    [['Qty', 'Form', 'QID', 'FOR', 'Duration', 'DurationUnit']]
    """
    
    tokens = sig.split()
    doc = nlp(' '.join(tokens))
    pos_tag = [token.tag_ for token in doc]
    sentence_triples = [(token, pos) for token, pos in zip(tokens, pos_tag)]
    features = [token_to_features(sentence_triples, i) for i in range(len(sentence_triples))]
    predictions = tagger.tag(features)
    
    print(sig)
    
    return [predictions]

### Sample predictions

In [85]:
predictions = predict("take 2 tabs every 6 hours x 10 days")
print(predictions)

take 2 tabs every 6 hours x 10 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']]


In [86]:
predictions = predict("2 capsu for 10 day at bed")
print(predictions)

2 capsu for 10 day at bed
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [87]:
predictions = predict("2 capsu for 10 days at bed")
print(predictions)

2 capsu for 10 days at bed
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [88]:
predictions = predict("5 days 2 tabs at bed")
print(predictions)

5 days 2 tabs at bed
[['Duration', 'DurationUnit', 'Qty', 'Form', 'AT', 'WHEN']]


In [89]:
predictions = predict("3 tabs qid x 10 weeks")
print(predictions)

3 tabs qid x 10 weeks
[['Qty', 'Form', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [90]:
predictions = predict("x 30 days")
print(predictions)

x 30 days
[['FOR', 'Duration', 'DurationUnit']]


In [91]:
predictions = predict("x 20 months")
print(predictions)

x 20 months
[['FOR', 'Duration', 'DurationUnit']]


In [92]:
predictions = predict("take 2 tabs po tid for 10 days")
print(predictions)

take 2 tabs po tid for 10 days
[['Method', 'Qty', 'Form', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit']]


In [93]:
predictions = predict("take 2 capsules po every 6 hours")
print(predictions)

take 2 capsules po every 6 hours
[['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit']]


In [94]:
predictions = predict("inject 2 units pu tid")
print(predictions)

inject 2 units pu tid
[['Method', 'Qty', 'Form', 'Frequency', 'TID']]


In [95]:
predictions = predict("swallow 3 caps tid by mouth")
print(predictions)

swallow 3 caps tid by mouth
[['Method', 'Qty', 'Form', 'TID', 'BY', 'PO']]


In [96]:
predictions = predict("inject 3 units orally")
print(predictions)

inject 3 units orally
[['Method', 'Qty', 'Form', 'Frequency']]


In [97]:
predictions = predict("orally take 3 tabs tid")
print(predictions)

orally take 3 tabs tid
[['Duration', 'DurationUnit', 'Qty', 'Form', 'TID']]


In [98]:
predictions = predict("by mouth take three caps")
print(predictions)

by mouth take three caps
[['BY', 'PO', 'Method', 'Qty', 'Form']]


In [99]:
predictions = predict("take 3 tabs orally three times a day for 10 days at bedtime")
print(predictions)

take 3 tabs orally three times a day for 10 days at bedtime
[['Method', 'Qty', 'Form', 'Frequency', 'Qty', 'TIMES', 'Period', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [100]:
predictions = predict("take 3 tabs orally bid for 10 days at bedtime")
print(predictions)

take 3 tabs orally bid for 10 days at bedtime
[['Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [101]:
predictions = predict("take 3 tabs bid orally at bed")
print(predictions)

take 3 tabs bid orally at bed
[['Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'AT', 'WHEN']]


In [102]:
predictions = predict("take 10 capsules by mouth qid")
print(predictions)

take 10 capsules by mouth qid
[['Method', 'Qty', 'Form', 'BY', 'PO', 'QID']]


In [103]:
predictions = predict("inject 10 units orally qid x 3 months")
print(predictions)

inject 10 units orally qid x 3 months
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [104]:
prediction = predict("please take 2 tablets per day for a month in the morning and evening each day")
print(predictions)

please take 2 tablets per day for a month in the morning and evening each day
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [105]:
prediction = predict("Amoxcicillin QID 30 tablets")
print(predictions)

Amoxcicillin QID 30 tablets
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [106]:
prediction = predict("take 3 tabs TID for 90 days with food")
print(predictions)

take 3 tabs TID for 90 days with food
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [107]:
prediction = predict("with food take 3 tablets per day for 90 days")
print(predictions)

with food take 3 tablets per day for 90 days
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [108]:
prediction = predict("with food take 3 tablets per week for 90 weeks")
print(predictions)

with food take 3 tablets per week for 90 weeks
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [109]:
prediction = predict("take 2-4 tabs")
print(predictions)

take 2-4 tabs
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [110]:
prediction = predict("take 2 to 4 tabs")
print(predictions)

take 2 to 4 tabs
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [111]:
prediction = predict("take two to four tabs")
print(predictions)

take two to four tabs
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [112]:
prediction = predict("take 2-4 tabs for 8 to 9 days")
print(predictions)

take 2-4 tabs for 8 to 9 days
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [113]:
prediction = predict("take 20 tabs every 6 to 8 days")
print(predictions)

take 20 tabs every 6 to 8 days
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [114]:
prediction = predict("take 2 tabs every 4 to 6 days")
print(predictions)

take 2 tabs every 4 to 6 days
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [115]:
prediction = predict("take 2 tabs every 2 to 10 weeks")
print(predictions)

take 2 tabs every 2 to 10 weeks
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [116]:
prediction = predict("take 2 tabs every 4 to 6 days")
print(predictions)

take 2 tabs every 4 to 6 days
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [117]:
prediction = predict("take 2 tabs every 2 to 10 months")
print(predictions)

take 2 tabs every 2 to 10 months
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [118]:
prediction = predict("every 60 mins")
print(predictions)

every 60 mins
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [119]:
prediction = predict("every 10 mins")
print(predictions)

every 10 mins
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [120]:
prediction = predict("every two to four months")
print(predictions)

every two to four months
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [121]:
prediction = predict("take 2 tabs every 3 to 4 days")
print(predictions)

take 2 tabs every 3 to 4 days
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [122]:
prediction = predict("every 3 to 4 days take 20 tabs")
print(predictions)

every 3 to 4 days take 20 tabs
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [123]:
prediction = predict("once in every 3 days take 3 tabs")
print(predictions)

once in every 3 days take 3 tabs
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [124]:
prediction = predict("take 3 tabs once in every 3 days")
print(predictions)

take 3 tabs once in every 3 days
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [125]:
prediction = predict("orally take 20 tabs every 4-6 weeks")
print(predictions)

orally take 20 tabs every 4-6 weeks
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [126]:
prediction = predict("10 tabs x 2 days")
print(predictions)

10 tabs x 2 days
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [127]:
prediction = predict("3 capsule x 15 days")
print(predictions)

3 capsule x 15 days
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [128]:
prediction = predict("10 tabs")
print(predictions)

10 tabs
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]
